## <span class="motutor-highlight motutor-id_vkzj6gr-id_mk7o8dt"><i></i>一个简单的 CNN 应用</span>

下面，我们将实现一个简单的 CNN 的网络，使用的框架为 Pytorch，这是一个由 FaceBook 推出的深度学习框架。我们会利用该框架搭建一个简答的 CNN 与 全连接网络的结合，然后实现手写数字的识别。首先我们介绍一下 在 Pytorch 中相关接口的说明：

二维卷积 nn.Conv2d()，该接口的输入为一个 $BatchSize \times Channels \times Height \times Width$ 的张量，注意，Pytorh 对图像/矩阵的存储是通道优先的，该接口的部分参数如下：
+ in_channels：输入矩阵/图像的通道数；
+ out_channels: 输出矩阵/图像的通道数，对应于卷积核的个数；
+ kernel_size: 卷积核的大小，kernel_size = 3 表示卷积核的大小为 $3 \times 3$，kernel_size=(3,5) 表示卷积核的大小为 $3 \times 5$；
+ stride：表示卷积核行进的步长，stride = 1 表示卷积核在图像的长宽维度上行进的步长均为 1，stride=(2,3) 表示在纵向行进的步长为 2，横向行进的步长为 3；
+ padding：图像周围的填充，通常补 0，默认不做 padding;
+ bias: 是否使用偏置，默认为 True。

全连接层 nn.Linear()，该接口的输入为 $BatchSize \times Features$，该接口的具体参数如下：
+ in_features: 表示该全连接层输入的节点个数；
+ out_features: 表示该全连接层输出的节点个数；
+ bias: 表示是否使用偏置，默认为 True。

In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        #self.bn1 = nn.
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [2]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
def main():
    BATCH_SIZE = 128
    epoches = 5
    lr = 1e-4
    
    torch.manual_seed(1)
    kwargs = {}
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True, **kwargs)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=BATCH_SIZE, shuffle=True, **kwargs)

    model = Net()
    optimizer = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(1, epoches + 1):
        train(model, train_loader, optimizer, epoch)
        test(model, test_loader)

    torch.save(model.state_dict(), "mnist_cnn.pt")
main()

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.306686

Test set: Average loss: 2.2861, Accuracy: 1500/10000 (15%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.283170

Test set: Average loss: 2.2699, Accuracy: 2258/10000 (23%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.267709

Test set: Average loss: 2.2530, Accuracy: 3107/10000 (31%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.256828
